In [1]:
import sys
sys.path.insert(1, r'C:\Users\Usuário\Documents\Github\Forex-Forecast\Notebooks')
from bb_setup import setup_1,main_features

In [2]:
import pandas as pd
import talib as ta
from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore")

In [7]:
pontos = 50
rate_stop = 1
rate_tp = 2
spread = 5
var_bb = 1.8

timeframe = 1

In [8]:
       
ativo = f'win_{timeframe}_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
df = df[(df['Data'] > '2022-11-01') & (df['Data'] < '2022-12-30')]

df_ticks = pd.read_csv('./Dados/tick_win_11_12.csv')

df = main_features(df)

df_acao = setup_1(df,df_ticks,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp,timeframe = timeframe)


In [176]:
#df_acao.to_csv('resultado_estrategia_1M_1_11_12.csv')
df_acao = pd.read_csv('resultado_estrategia_1M_1_11_12.csv')

In [177]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')

Timeframe: 1 --pontos: 50 --- rate_tp: 2 -- var_bb: 1.8
Pontos_liquido: 4800.0 -- Total_operacoes: 990
Agrupado por Hora
  Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0     09             88     0.386364           260.0
1     10            115     0.478261          1925.0
2     11            105     0.380952           225.0
3     12            102     0.421569           840.0
4     13            108     0.398148           510.0
5     14            105     0.457143          1425.0
6     15            102     0.372549            90.0
7     16            108     0.435185          1110.0
8     17             99     0.363636           -45.0
9     18             32     0.343750          -110.0
Agrupado por Mes
  Mes  qtd_operacoes  taxa_acerto  pontos_liquido
0  11            532     0.402256          2840.0
1  12            432     0.418981          3390.0



#### Modelo

In [160]:
#df_acao = df_acao[df_acao['Hora_h'].isin(['09','10','11','12','13','14','15','16'])]

In [285]:
df_modelo = df_acao[['Data','acao','Vol','macdhist','dist_bb','dist_bb_up','dist_bb_low','dist_media','dist_media_abs',
'dist_price_media_21','dist_price_media_5','dist_RSI_70','dist_RSI_30','tamanho_corpo','tamanho_full',
'media_vol_5','media_vol_3','media_vol_10','tamanho_corpo_5','tamanho_corpo_direcao','candle_direcao','candle_direcao_5','resultado_binario']]

In [207]:
df_modelo = df_acao[['Data','acao','macdhist','media_vol_3','media_vol_5','candle_direcao_5','resultado_binario']]

In [264]:
df_modelo = df_modelo[df_modelo['acao'] == 'sell']

In [299]:
inicio = '2022-11-01'
termino = '2022-11-30'
df_modelo = df_modelo.dropna()

df_train = df_modelo[(df_modelo['Data'] > inicio) & (df_modelo['Data'] < termino)]
df_train = df_train.set_index(['Data','acao'])
#df_train=(df_train-df_train.min())/(df_train.max()-df_train.min())

df_1 = df_train[df_train['resultado_binario'] == 1]
df_0 = df_train[df_train['resultado_binario'] == 0].iloc[:df_1.shape[0]-100]

df_train = pd.concat([df_1,df_0])
df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test = df_modelo[(df_modelo['Data'] > termino)]
df_test = df_test.set_index(['Data','acao'])
#df_test=(df_test-df_test.min())/(df_test.max()-df_test.min())

y_train = df_train['resultado_binario']
x_train = df_train.drop(columns=['resultado_binario'])

y_test = df_test['resultado_binario']
x_test = df_test.drop(columns=['resultado_binario'])

In [300]:
df_train['resultado_binario'].sum()/df_train.shape[0]

0.6633986928104575

In [301]:
df_test['resultado_binario'].sum()/df_test.shape[0]

0.4170305676855895

In [302]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import lightgbm as lgb

In [306]:
#clf = ExtraTreesClassifier()
clf = XGBClassifier()
#clf = lgb.LGBMClassifier()

clf.fit(x_train, y_train)

'''clf = XGBClassifier()
clf.load_model('xgboost_1M_estrategia1.txt')'''

# predict the results
y_pred=clf.predict(x_test)
y_proba = clf.predict_proba(x_test)[:,1]
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

print('Training set score: {:.4f}'.format(clf.score(x_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(x_test, y_test)))

LightGBM Model accuracy score: 0.4803
Training set score: 1.0000
Test set score: 0.4803


In [307]:
'''import joblib
#save model
joblib.dump(clf, 'xgboost_1M.txt')'''

"import joblib\n#save model\njoblib.dump(clf, 'xgboost_1M.txt')"

In [308]:
df_thr = x_test.copy()
df_thr = df_thr.reset_index()
df_thr['pred'] = y_pred
df_thr['proba'] = y_proba
df_thr['resultado_binario'] = y_test.values

lista_full = []
for thr in [0.5,0.6,0.7,0.8,0.9]:

    df_temp = df_thr[df_thr['proba'] > thr]

    qtd_operacoes = df_temp.shape[0]

    acc = df_temp[df_temp['resultado_binario'] ==1].shape[0]/qtd_operacoes



    lista_full.append([thr,qtd_operacoes,acc])

df_a = pd.DataFrame(lista_full)
df_a.columns = ['thr','qtd_operacoes','acc']
df_a

,thr,qtd_operacoes,acc
0,0.5,335,0.429851
1,0.6,315,0.419048
2,0.7,285,0.424561
3,0.8,242,0.438017
4,0.9,186,0.430108


In [513]:
#clf.save_model('xgboost_1M_estrategia1.model')

In [195]:
pd.Series(clf.feature_importances_, index=x_test.columns).nlargest(20)

macdhist                 0.060004
media_vol_10             0.058552
media_vol_3              0.057620
candle_direcao_5         0.056469
dist_bb_low              0.056420
tamanho_corpo_5          0.055724
Vol                      0.054471
dist_media_abs           0.054253
dist_media               0.052970
media_vol_5              0.051754
tamanho_corpo_direcao    0.050299
dist_price_media_5       0.050261
dist_price_media_21      0.048917
tamanho_full             0.048584
dist_bb                  0.048334
tamanho_corpo            0.047788
dist_RSI_30              0.047753
dist_bb_up               0.047282
dist_RSI_70              0.045847
candle_direcao           0.006698
dtype: float64

In [259]:
df_acao_nova = df_thr[df_thr['proba'] > 0.9]

In [260]:
qtd_operacoes = df_acao_nova.shape[0]
taxa_acerto = df_acao_nova['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao_nova['Hora'] = df_acao_nova['Data'].apply(lambda x: x[11:])
df_acao_nova['Hora_h'] = df_acao_nova['Hora'].apply(lambda x: x[:2])
df_acao_nova['Mes'] = df_acao_nova['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao_nova[['Mes','resultado_binario']]
df_new_hora = df_acao_nova[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')

Timeframe: 1 --pontos: 50 --- rate_tp: 2 -- var_bb: 1.8
Pontos_liquido: 1775.0 -- Total_operacoes: 85
Agrupado por Hora
  Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0     09              4     0.250000           -70.0
1     10             15     0.533333           375.0
2     11              6     0.666667           270.0
3     12              9     0.777778           555.0
4     13              9     0.444444           105.0
5     14             10     0.600000           350.0
6     15              8     0.500000           160.0
7     16             11     0.545455           295.0
8     17              5     0.200000          -125.0
9     18              8     0.250000          -140.0
Agrupado por Mes
  Mes  qtd_operacoes  taxa_acerto  pontos_liquido
0  12             85     0.505882          1775.0



In [ ]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')

In [547]:
x_test.columns

Index(['Vol', 'macdhist', 'dist_bb', 'dist_bb_up', 'dist_bb_low', 'dist_media',
       'dist_media_abs', 'dist_price_media_21', 'dist_price_media_5',
       'dist_RSI_70', 'dist_RSI_30', 'tamanho_corpo', 'tamanho_full',
       'media_vol_5', 'media_vol_3', 'media_vol_10', 'tamanho_corpo_5',
       'tamanho_corpo_direcao', 'candle_direcao', 'candle_direcao_5'],
      dtype='object')